In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import rasterio
from scipy.ndimage import binary_closing, binary_fill_holes, label
from matplotlib.patches import Patch

In [ ]:
imagens_TIF = '../../Dados/Imagens/'

caminho_para_arquivos = []
path_obj = Path(imagens_TIF)
for cidade in path_obj.iterdir():
    if(cidade.is_dir() and 'Precisao' not in cidade.name):
        caminho_para_arquivos.append(f'{imagens_TIF}{cidade.name}')

In [ ]:
def lerArquivo(arquivo):
    with rasterio.open(arquivo) as banda_src:
        img = banda_src.read(1)
        profile = banda_src.profile
    return img, profile

In [ ]:
def criarELerRGB(b2,b3,b4):
    with rasterio.open(b4) as r, \
          rasterio.open(b3) as g, \
          rasterio.open(b2) as b:
            red = r.read(1)
            green = g.read(1)
            blue = b.read(1)
            rgb = np.stack([red, green, blue], axis=-1)
            p2 = np.percentile(rgb, 2)
            p98 = np.percentile(rgb, 98)
            rgb = np.clip((rgb - p2) / (p98 - p2), 0, 1)
    return rgb

In [ ]:
def lerArquivoRGB(arquivo):
    try:
        with rasterio.open(arquivo) as banda_src:            
            img = banda_src.read([1, 2, 3])
            rgb = np.transpose(img, (1, 2, 0))
            rgb /= rgb.max()
    except Exception as ex:
        print(f"Falha ao ler imagem RGB: {ex}")
        b2 = str(arquivo).replace("True_color", "B02_(Raw)")
        b3 = str(arquivo).replace("True_color", "B03_(Raw)")
        b4 = str(arquivo).replace("True_color", "B04_(Raw)")
   
        if not all([b2, b3, b4]):
            raise ValueError("Bandas B2, B3 e B4 são necessárias")
        rgb = criarELerRGB(b2,b3,b4)
    return rgb

In [ ]:

def salvarArquivo(dados, profile, caminho, nome):
    saida = os.path.join(caminho,  f"{nome}.tif")
    profile.update(dtype='float32', nodata=0)
    with rasterio.open(saida, 'w', **profile) as dst:
        dst.write(dados, 1)
       
def removerNuvens(cloud_mask, banda):
    return np.where(cloud_mask, np.nan, banda)

In [ ]:
def plotCategorias(img):
    classes = {
        0: 'No Data',
        1: 'Saturated or defective pixel',
        2: 'Topographic casted shadows',
        3: 'Cloud Shadows',
        4: 'Vegetation',
        5: 'Not vegetated',
        6: 'Water',
        7: 'Unclassified',
        8: 'Clouds medium probability',
        9: 'Clouds high probability',
        10: 'Thin Cirrus',
        11: 'Snow / Ice'
    }

    classes_presentes = {k: v for k, v in classes.items() if k in np.unique(img)}

    colors = [
        '#000000',  # 0 - Fundo
        '#7f7f7f',  # 1 - Não classificado
        '#6e6e6e',  # 2 - Sombra de nuvem
        '#99cc66',  # 3 - Vegetação baixa
        '#336633',  # 4 - Vegetação alta
        '#d2b48c',  # 5 - Solo nu
        '#3399ff',  # 6 - Água
        '#ffcc66',  # 7 - Nuvem média probabilidade
        '#ff9933',  # 8 - Nuvem alta probabilidade
        '#cce6ff',  # 9 - Cirros
        '#4d4d4d',  # 10 - Sombra de nuvem
        "#f3f3c9"   # 11 - Neve ou gelo
    ]
    cmap = mcolors.ListedColormap(colors)

    # Ajuste os limites para o BoundaryNorm
    boundaries = np.arange(-0.5, len(colors) + 0.5, 1)
    norm = mcolors.BoundaryNorm(boundaries, ncolors=len(colors))

    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(img, cmap=cmap, norm=norm)
    # Legenda
    handles = []
    for value, label in classes_presentes.items():
        handles.append(
            plt.Line2D([0], [0], marker='s', color='w', label=f"{value}: {label}",
                    markerfacecolor=cmap(norm(value)), markersize=15)
        )

    ax.legend(handles=handles, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    ax.set_title('Scene Classification Map')
    ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
def plotarMascaraECategorias(cloud_mask, cloud_mask_buracos_fechados, cloud_mask_preenchida):
    fig, axs = plt.subplots(1,3,figsize=(20, 20))
    axs[0].imshow(cloud_mask)
    axs[0].set_title('mascara')
    axs[0].axis('off')
    
    axs[1].imshow(cloud_mask_buracos_fechados)
    axs[1].set_title('fechado')
    axs[1].axis('off')
    
    axs[2].imshow(cloud_mask_preenchida)
    axs[2].set_title('preenchido')
    axs[2].axis('off')

In [ ]:
def aplicaCloudMask(arquivo):
        with rasterio.open(arquivo) as banda_src:
                img = banda_src.read(1)

        img_classes = np.round(img * 11).astype(int)
        
        plotCategorias(img_classes)
        cloud_mask = np.isin(img_classes, [3,8,9,10])
        matriz = np.ones((2,2)) 
        cloud_mask_buracos_fechados = binary_closing(cloud_mask, structure=matriz)

        cloud_mask_preenchida = binary_fill_holes(cloud_mask_buracos_fechados, structure=matriz)
        plotarMascaraECategorias(cloud_mask, cloud_mask_buracos_fechados, cloud_mask_preenchida)
        return cloud_mask_preenchida

In [ ]:
def rgbTiraMascara(rgb, cloud_mask):
    rgbMask = rgb.copy()
    rgbMask[cloud_mask == 1] = np.nan
    return rgbMask

In [ ]:

def plotarTodos(tc):
    fig, axs = plt.subplots(1,3, figsize=(20, 20))
    for index, value in enumerate(tc):
        axs[index].imshow(value)
        axs[index].axis('off')
    plt.show()
    
def plotMedia(media, nome):
    fig, axs = plt.subplots(figsize=(20, 20))
    plt.imshow(media)
    plt.axis('off')
    plt.title(nome)
    plt.show()
    
    
def criaMedia(caminho, nome, profile, imagens):
    pilha = np.stack(imagens, axis=0)
    media = np.nanmean(pilha, axis=0)
    if(profile is not None):
        salvarArquivo(media, profile, caminho, nome)
    else:
        plotMedia(media, nome)
    

In [ ]:

subpastas = ['Janeiro', 'Fevereiro', 'Março']
for cidade_path in caminho_para_arquivos:
    print(cidade_path)
    b3_sem_mascara = []
    b4_sem_mascara = []
    b8_sem_mascara = []
    tc = []
    tc_sem_mascara = []

    cloud_mask = []
    b3 = []
    b4 = []
    b8 = []

    for mes in subpastas:
        profile = None
        pasta_mes = os.path.join(cidade_path, mes)
        arquivos = os.listdir(pasta_mes)
        for arquivo in arquivos:
            caminho_completo = os.path.join(pasta_mes, arquivo)
            if "B03" in arquivo:
                img, profile = lerArquivo(caminho_completo)
                b3.append(img)
            elif "B04" in arquivo:
                img, _ = lerArquivo(caminho_completo)
                b4.append(img)
            elif "B08" in arquivo:
                img, _ = lerArquivo(caminho_completo)
                b8.append(img)
            elif "Scene_classification_map" in arquivo:
                cloud_mask.append(aplicaCloudMask(caminho_completo))
            elif "True_color" in arquivo:
                tc.append(lerArquivoRGB(caminho_completo))

    if len(cloud_mask) == 3:
        for index, mask in enumerate(cloud_mask):
            tc_sem_mascara.append(rgbTiraMascara(tc[index], mask))
            b3_sem_mascara.append(removerNuvens(mask, b3[index]))
            b4_sem_mascara.append(removerNuvens(mask, b4[index]))
            b8_sem_mascara.append(removerNuvens(mask, b8[index]))

    plotarTodos(tc_sem_mascara)
    if tc_sem_mascara != []:
        criaMedia(cidade_path, os.path.basename(cidade_path), None, tc_sem_mascara)
    if b8_sem_mascara and profile is not None:
        criaMedia(cidade_path, 'B8', profile, b8_sem_mascara)
    if b4_sem_mascara and profile is not None:
        criaMedia(cidade_path, 'B4', profile, b4_sem_mascara)
    if b3_sem_mascara and profile is not None:
        criaMedia(cidade_path, 'B3', profile, b3_sem_mascara)

In [ ]:
def calcularIndices(banda_x, banda_y, caminho, profile, nome):
    denominador = banda_x + banda_y
    denominador[denominador == 0] = 1e-6
    indice = (banda_x - banda_y)/(denominador)
    salvarArquivo(indice, profile, caminho, nome)       

for cidade_path in caminho_para_arquivos:
    b3, profile = lerArquivo(os.path.join(cidade_path, 'B3.tif'))
    b4, _ = lerArquivo(os.path.join(cidade_path, 'B4.tif'))
    b8, _ = lerArquivo(os.path.join(cidade_path, 'B8.tif'))
    
    calcularIndices(b8, b4, cidade_path, profile, "NDVI")
    calcularIndices(b3, b8, cidade_path, profile, "NDWI")

In [ ]:
def criaImagemIndices(ndvi, ndwi, nome):
    class_mask = np.zeros_like(ndvi, dtype=np.uint8)

    class_mask[ndwi > 0.3] = 1
    class_mask[(ndvi > 0.5) & (ndwi <= 0.3)] = 2 

    colors = np.array([
        [255, 255, 255],  # Outro - branco
        [0, 0, 255],      # Água - azul
        [0, 128, 0],      # Vegetação - verde escuro
    ], dtype=np.uint8)
    class_rgb = colors[class_mask]

    fig, axs = plt.subplots( figsize=(12, 6))

    plt.imshow(class_rgb)
    plt.title(f"{nome} -  Água e Vegetação")
    plt.axis("off")
    legend_elements = [
        Patch(facecolor=np.array([0, 0, 255])/255, label='Água'),
        Patch(facecolor=np.array([0, 128, 0])/255, label='Vegetação'),
        Patch(facecolor=np.array([255, 255, 255])/255, label='Outro'),
    ]
    axs.legend(handles=legend_elements, loc='lower right')

    plt.tight_layout()
    plt.show()

In [ ]:

for cidade_path in caminho_para_arquivos:
    ndvi, _ = lerArquivo(os.path.join(cidade_path, 'NDVI.tif'))
    ndwi, _ = lerArquivo(os.path.join(cidade_path, 'NDWI.tif'))
    criaImagemIndices(ndvi, ndwi, os.path.basename(cidade_path))